In [1]:
import os
import json
import re
import pandas as pd

from datasets import load_dataset

/opt/anaconda/envs/diff-direction/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
ds = load_dataset("openai/gsm8k", "main")

In [25]:
for ds_split in ds.keys():
    extracted_answer_list = [item["answer"].split("\n####")[-1].strip() for item in ds[ds_split]]
    ds[ds_split] = ds[ds_split].map(lambda x, idx: {"answer": extracted_answer_list[idx]}, with_indices=True)

Map: 100%|██████████| 1319/1319 [00:00<00:00, 32910.49 examples/s]


In [40]:
train_ds = ds["train"]
train_ds.to_pandas()

,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,72
1,Weng earns $12 an hour for babysitting. Yester...,10
2,Betty is saving money for a new wallet which c...,5
3,"Julie is reading a 120-page book. Yesterday, s...",42
4,James writes a 3-page letter to 2 different fr...,624
...,...,...
7468,"Very early this morning, Elise left home in a ...",5
7469,Josh is saving up for a box of cookies. To rai...,3
7470,Colin can skip at six times the speed that Bra...,4
7471,"Janet, a third grade teacher, is picking up th...",308


In [33]:
df = pd.read_parquet("../hard_rl/data/MATH/train.parquet")

In [36]:
df["prompt"].iloc[3]

array([{'content': 'Alice wants to buy $3$ pounds of veal at the grocery store, but the scales at the store only show weight in kilograms. If one kilogram is $2.20$ pounds, how many kilograms of veal should Alice buy?  (You may use a calculator on this problem; answer to the nearest hundredth.) Let\'s think step by step and output the final answer after "\\boxed{}".', 'role': 'user'}],
      dtype=object)

In [ ]:
df_2 = pd.read_json("../predicting_learnability/data/SR_DATASETS/MATH/validation.jsonl", lines=True)

In [49]:
df_test = pd.read_parquet("../predicting_learnability/data/SR_DATASETS/GSM8K/Qwen2.5-Math-1.5B-Instruct-SR_train_max_3000_k_1_temp_0.0/Qwen2.5-Math-1.5B-Instruct-SR_train_max_3000_k_1_temp_0.0.parquet")

In [51]:
df_test["prompt"].iloc[0]

"<|im_start|>system\nPlease reason step by step, and put your final answer within \\boxed{}.<|im_end|>\n<|im_start|>user\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? Let's think step by step and output the final answer after \\boxed{}.<|im_end|>\n<|im_start|>assistant\n"

In [52]:
from transformers import AutoTokenizer

In [53]:
tok = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-1.5B-Instruct")